In [1]:
# 导入库
import time  
import numpy as np  
import pandas as pd 
from datetime import datetime

In [2]:
# 基本状态查看
def stats_summary(df):
    '''
    查看数据集的记录数、维度数、前2条数据、描述性统计和数据类型
    :param df: 数据框
    :return: 无
    '''
    print('Data Overview:')
    print('Records: {0}\tDimension：{1}'.format(df.shape[0], df.shape[1]))  # 打印数据形状
    print('-' * 30)
    print('The first two rows:')
    print(df.head(2))    # 输出前2条数据
    print('-' * 30)
    print('Data Desc:')
    print(df.describe(include='all'))  # 输出数据描述性统计信息
    print('-' * 30)
    print('Data Dtypes:')
    print(df.dtypes)      # 输出数据类型
    print('-' * 60)

In [3]:
# 读取数据文件
raw_data = pd.read_csv('sales.csv')  
stats_summary(raw_data)

Data Overview:
Records: 86135	Dimension：4
------------------------------
The first two rows:
   USERID   ORDERDATE     ORDERID  AMOUNTINFO
0  142074  2016-01-01  4196439032      9399.0
1   56927  2016-01-01  4198324983      8799.0
------------------------------
Data Desc:
               USERID   ORDERDATE       ORDERID    AMOUNTINFO
count    86135.000000       86133  8.613500e+04  86127.000000
unique            NaN         347           NaN           NaN
top               NaN  2016-04-18           NaN           NaN
freq              NaN         300           NaN           NaN
mean    108059.642782         NaN  4.137051e+09    744.705249
std      32866.109087         NaN  1.238664e+08   1425.211176
min      51220.000000         NaN  3.002478e+09      0.500000
25%      79565.500000         NaN  4.114637e+09     13.000000
50%     108072.000000         NaN  4.190615e+09     59.000000
75%     136378.000000         NaN  4.196161e+09    629.000000
max     165100.000000         NaN  4.203330e+

In [5]:
# 缺失值查看
na_lines = raw_data.isnull().any(axis=1)  # 查看每一行是否具有缺失值
print('Total number of NA lines is: {0}'.format(na_lines.sum()))  # 查看具有缺失值的行总记录数
print(raw_data[na_lines])  # 只查看具有缺失值的行信息

# 重复值查看
print('-' * 30)
print('Total number of duplicated lines is: {0}'.format(raw_data.duplicated().sum()))  #重复出现的行记录（排除第一次出现）

# 缺失值丢弃处理
sales_data = raw_data.dropna()  # 丢弃带有缺失值的行记录

# 异常值丢弃处理
sales_data = sales_data[sales_data['AMOUNTINFO'] > 1]  # 丢弃订单金额<=1的记录

print(sales_data.shape)

Total number of NA lines is: 10
       USERID   ORDERDATE     ORDERID  AMOUNTINFO
20      75849  2016-01-01  4197103430         NaN
72     103714         NaN  4136159682       189.0
114    155209  2016-01-01  4177940815         NaN
229    139877         NaN  4111956196         6.3
233     54599  2016-01-01  4119525205         NaN
260     65456  2016-01-02  4195643356         NaN
62134  122134  2016-09-21  3826649773         NaN
70342  116995  2016-10-24  3981569421         NaN
80110   98888  2016-12-06  3814398698         NaN
86069  145951  2016-12-29  4139830098         NaN
------------------------------
Total number of duplicated lines is: 0
(84342, 4)


In [6]:
# 日期格式转换
sales_data['ORDERDATE'] = pd.to_datetime(sales_data['ORDERDATE'], format='%Y-%m-%d')  # 将字符串转换为日期格式
print(max(sales_data['ORDERDATE']),min(sales_data['ORDERDATE']))

2016-12-29 00:00:00 2016-01-01 00:00:00


In [7]:
# 数据转换
recency_value = sales_data['ORDERDATE'].groupby(sales_data.USERID).max()      # 计算每用户最近一次订单时间
frequency_value = sales_data['ORDERDATE'].groupby(sales_data.USERID).count()  # 计算每用户订单频率
monetary_value = sales_data['AMOUNTINFO'].groupby(sales_data.USERID).sum()    # 计算每用户订单总金额

# 分别计算R、F、M得分
deadline_date = datetime(2017, 1, 1)  # 指定一个时间节点，用于计算其他时间与该时间的距离
r_interval = (deadline_date - recency_value).dt.days  # 计算R间隔
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1])       # 计算R得分，等距分段
f_score = pd.cut(frequency_value, 5, labels=[1, 2, 3, 4, 5])  # 计算F得分，等距分段
m_score = pd.cut(monetary_value, 5, labels=[1, 2, 3, 4, 5])   # 计算M得分，等距分段

# R、F、M数据合并
rfm_list = [r_score, f_score, m_score]  
rfm_cols = ['r_score', 'f_score', 'm_score']  
rfm_df = pd.DataFrame(np.array(rfm_list).transpose(), columns=rfm_cols,
                      index=recency_value.index)  

print('RFM Separate Score Overview:')
print(rfm_df.head())

RFM Separate Score Overview:
        r_score  f_score  m_score
USERID                           
51220         4        1        1
51221         2        1        1
51224         3        1        1
51225         4        1        1
51226         1        1        1


In [8]:
# 计算RFM总得分
# 方法一：加权得分
rfm_df['rfm_wscore'] = rfm_df['r_score'] * 0.4 + rfm_df['f_score'] * 0.4 + rfm_df['m_score'] * 0.2
# 方法二：RFM组合
rfm_df_tmp = rfm_df.copy()
rfm_df_tmp['r_score'] = rfm_df_tmp['r_score'].astype(str)
rfm_df_tmp['f_score'] = rfm_df_tmp['f_score'].astype(str)
rfm_df_tmp['m_score'] = rfm_df_tmp['m_score'].astype(str)
rfm_df['rfm_comb'] = rfm_df_tmp['r_score'].str.cat(rfm_df_tmp['f_score']).str.cat(rfm_df_tmp['m_score'])

print('RFM Total Score Overview :')
print(rfm_df.head())

# 保存RFM得分到本地文件
rfm_df.to_csv('sales_rfm_score.csv')  # 保存数据为csv

RFM Total Score Overview :
        r_score  f_score  m_score  rfm_wscore rfm_comb
USERID                                                
51220         4        1        1         2.2      411
51221         2        1        1         1.4      211
51224         3        1        1         1.8      311
51225         4        1        1         2.2      411
51226         1        1        1         1.0      111
